<a href="https://colab.research.google.com/github/MuhamedZepcanin/MuhamedZepcanin.github.io/blob/main/pokemongame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import random
import sys

pygame.init()
pygame.mixer.init()

WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pokémon Battle")

clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 28)
big_font = pygame.font.SysFont(None, 40)

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (200, 50, 50)
GREEN = (50, 200, 50)

# Load images
pokemon_images = {
    "Pikachu": pygame.image.load("images/pikachu.png"),
    "Bulbasaur": pygame.image.load("images/bulbasaur.png"),
    "Charmander": pygame.image.load("images/charmander.png")
}

bg = pygame.image.load("images/battle_bg.png")
bg = pygame.transform.scale(bg, (WIDTH, HEIGHT))


class Pokemon:
    def __init__(self, name, hp, attack):
        self.name = name
        self.level = 1
        self.max_hp = hp
        self.hp = hp
        self.attack = attack
        self.xp = 0

    def level_up(self):
        self.level += 1
        self.max_hp += 10
        self.attack += 3
        self.hp = self.max_hp



party = [Pokemon("Pikachu", 100, 20)]
active_index = 0


def draw_text(text, x, y, big=False):
    img = (big_font if big else font).render(text, True, BLACK)
    screen.blit(img, (x, y))


def draw_hp_bar(x, y, p):
    ratio = max(p.hp / p.max_hp, 0)
    pygame.draw.rect(screen, RED, (x, y, 200, 20))
    pygame.draw.rect(screen, GREEN, (x, y, 200 * ratio, 20))


def shake(sprite, base_x, y):
    for _ in range(6):
        screen.blit(bg, (0, 0))
        screen.blit(sprite, (base_x + random.randint(-5, 5), y))
        pygame.display.update()
        pygame.time.delay(40)


def battle_text(msg):
    pygame.draw.rect(screen, WHITE, (50, 460, 700, 80))
    pygame.draw.rect(screen, BLACK, (50, 460, 700, 80), 2)
    draw_text(msg, 70, 490)
    pygame.display.update()
    pygame.time.delay(900)


def battle():
    global active_index

    enemy = Pokemon(
        random.choice(["Bulbasaur", "Charmander"]),
        random.randint(80, 110),
        random.randint(15, 20)
    )

    # Slide-in animation
    player_x = -200
    enemy_x = WIDTH + 200

    while player_x < 100:
        screen.blit(bg, (0, 0))
        screen.blit(pokemon_images[party[active_index].name], (player_x, 320))
        screen.blit(pokemon_images[enemy.name], (enemy_x, 120))
        player_x += 15
        enemy_x -= 15
        pygame.display.update()
        clock.tick(60)

    battle_text(f"A wild {enemy.name} appeared!")

    while True:
        screen.blit(bg, (0, 0))
        active = party[active_index]

        screen.blit(pokemon_images[active.name], (100, 320))
        screen.blit(pokemon_images[enemy.name], (550, 120))

        draw_text(active.name, 100, 280)
        draw_hp_bar(100, 305, active)

        draw_text(enemy.name, 550, 80)
        draw_hp_bar(550, 105, enemy)

        draw_text("A:Attack  C:Catch  1–6:Switch  P:Pokedex", 140, 560)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.KEYDOWN:

                if event.key == pygame.K_a:

                    enemy.hp -= random.randint(active.attack - 5, active.attack + 5)
                    shake(pokemon_images[enemy.name], 550, 120)
                    battle_text(f"{active.name} attacked!")

                    if enemy.hp > 0:
                        active.hp -= random.randint(enemy.attack - 4, enemy.attack + 4)
                        shake(pokemon_images[active.name], 100, 320)

                if event.key == pygame.K_c:

                    if random.random() < 0.5 and len(party) < 6:
                        party.append(enemy)
                        battle_text(f"{enemy.name} was caught!")
                        return
                    else:
                        battle_text("The Pokémon broke free!")
                        active.hp -= enemy.attack

                if pygame.K_1 <= event.key <= pygame.K_6:
                    idx = event.key - pygame.K_1
                    if idx < len(party) and party[idx].hp > 0:
                        active_index = idx
                        battle_text(f"Go {party[idx].name}!")

        if enemy.hp <= 0:
            active.xp += 20
            battle_text(f"{enemy.name} fainted!")
            if active.xp >= 50:
                active.xp = 0
                active.level_up()
            return

        if active.hp <= 0:
            alive = [i for i, p in enumerate(party) if p.hp > 0]
            if alive:
                active_index = alive[0]
                battle_text("Switching Pokémon!")
            else:
                battle_text("All Pokémon fainted!")
                pygame.quit()
                sys.exit()

        pygame.display.update()
        clock.tick(60)


while True:
    battle()
